#  Uncertainty Quantification for Economists

## Abbreviations

UQ - Uncertainty Quantification  
QoI - Quantity of Interest

## 1. Introduction

Uncertainty Quantification is an increasingly well-established toolkit in many applied sciences such as metereology, climatology, engineering, virology etc. Models to which the tools of UQ are applied are amongst others weather and climate models, nuclear reactor models and HIV models. Applioed UQ aims to analyze the uncertainty of mathematical models that may come from various sources. The sources can be categorized in (i) model errors or discrepancies, (ii) input uncertainties, (iii) numerical errors and uncertainties and (iv) measurement errors and uncertainties. Once they are identified, the model can be improved. Given the magnitude of uncertainty in most models that are used to make - oftentimes tremendously important - decisions, Uncertainty Quantification should be part of every serious applied model-based research and practice.

Usually, the second source of uncertainty, input uncertainties, is the main objective for an applied UQ. The reasons are first, that the tools to identify the model uncertainty that is propagated by the input paramters are relatively accessible and second, that this uncertainaty is oftentimes hard to resolve and thus should at least be reported. The following shows the steps to be taken when applying UQ to some model.

![](_notebook_material/uq_framework.png)

In Step A, the computational model $M$ is defined. This requires to identify input paramters and putput quantities of interest.  
In Step B, the uncertainty in the input paramters is described by a joint PDF that best represents the available information.  
In Step C, uncertainty propagation is carried out so as to analyze the moments and dsitributions of the QoI, for instance by plotting histograms.  
Finally, in Step C', sensitivity analysis is carried out to rank the input parameters according to their impact onto the predicition uncertainty, or to identify the direction of change of that impact. The acquired knowledge can be used to focus a more detailed analysis on a subset of the input variables.



## 2. UQ Economics

## 3. Our economic models: An example

## 4. Our learning interests

## References

Feinberg, J., Langtagen, H. P. (2015) ["Chaospy: An open source tool for designing methods of uncertainty quantification"](https://www.sciencedirect.com/science/article/pii/S1877750315300119), Journal of Computational Science, 11: pp. 46-57.

Harenberg, D., Marelli, S., Sudret B., Winschel V. (2019). ["Uncertainty quantification and global sensitivity analysis for economic models"](https://onlinelibrary.wiley.com/doi/abs/10.3982/QE866), Quantitative Economics, 10(1): 1-41.

Keane, M. P. & Wolpin, W. I. (1994). ["The Solution and Estimation of Discrete Choice Dynamic Pogramming Models by Simulation and Interpolation: Monte Carlo Evidence"](http://research.economics.unsw.edu.au/mkeane/Solution_Estimation_DP.pdf), Review of Economics and Statistics, 76(4): pp. 648-672.

Keane, M. P. and Wolpin, K. I. (1997). ["The Career Decisions of Young Men"](https://www.jstor.org/stable/pdf/10.1086/262080.pdf?refreqid=excelsior%3A4e16df222bafe665c89727fac7aad557&seq=1#page_scan_tab_contents). Journal of Political Economy, 105(3): pp. 473-522.

Smith, R. C. (2013). [Uncertainty Quantification: Theory, Implementation, and Applications](https://my.siam.org/Store/Product/viewproduct/?ProductId=24973024). SIAM. Philadelphia, Pennsylvania.